# Gather unWISE photometry for the HST sample

The goal of this notebook is to document how we retrieve DR14 SDSS *ugriz* and unWISE *W1-W2* (forced) photometry for the HST HizEA sample of galaxies.

The input catalog is
* $HIZEA_DIR/etc/hst_sample.dat

and the output (row-matched) catalog containing *just* the SDSS/unWISE photometry is
* $HIZEA_DIR/etc/XXX

In [1]:
import os
import numpy as np

from astropy.io import ascii
from astropy.table import Table

### [1] Read the sample and write out a temporary file we can upload to CasJobs.

In [2]:
def write_casjobs_file():
    outfile = os.path.join(os.sep, 'Users', 'ioannis', 'Desktop', 'hizea_sample.tmp')
    dat = ascii.read( os.path.join( os.getenv('HIZEA_DIR'), 'etc', 'hst_sample.dat' ) )
    out = Table()
    out['hizea_id'] = np.arange(len(dat))
    out['hizea_ra'] = dat['ra']
    out['hizea_dec'] = dat['dec']
    print('Writing {}'.format(outfile))
    out.write(outfile, format='ascii.commented_header', overwrite=True)
    return out

In [3]:
out = write_casjobs_file()
out

Writing /Users/ioannis/Desktop/hizea_sample.tmp


hizea_id,hizea_ra,hizea_dec
int64,float64,float64
0,226.65123962,54.03909907
1,136.3483394,57.98680093
2,205.40333219,-3.35701866
3,146.07436936,9.50537816
4,325.00205248,12.15405848
5,126.66007153,43.09150738
6,243.38552761,28.5707732
7,184.98241233,3.60441687
8,166.7619674,4.28410187


### [2] Import the sample into CasJobs.

Unfortunately when you import a simple ASCII file, CasJobs does not keep the RA,Dec coordinates in double precision.  So first create a table in the *Query* tab (using the *Quick* submission option) thusly:
```SQL
CREATE TABLE hizea ( hizea_id int, hizea_ra float, hizea_dec float );
```
and then in the *Import* tab be sure to import the data directly into this table.

### [3] Search for ObjID using RA,Dec positions.

Next, gather the SDSS ObjID using the RA,Dec positions and a 1 arcsec search radius.  In the *Query* tab be sure to set the *Context* to DR14 and then execute the following query.

```SQL
SELECT
  h.hizea_id,
  h.hizea_ra,
  h.hizea_dec,
  p.objid,
  p.ra,
  p.dec,
  p.run,
  p.rerun,
  p.camcol
INTO mydb.hizea_match FROM mydb.hizea AS h
  OUTER APPLY dbo.fGetNearestObjEq( h.hizea_ra, h.hizea_dec, 0.01666) AS n 
  LEFT JOIN dr14.PhotoObj AS p ON n.objid=p.objid
ORDER BY h.hizea_id
```

### [4] Finally gather the SDSS + unWISE photometry.

Finally, we use the output of the previous step to retrieve the full set of SDSS (*ugriz*) and unWISE (*W1W2*) forced photometry by executing the following query (again, using the DR14 *Context*).  Note that we call the output table (not creatively) *hst_sample_sdssWISEphot.fits*.

```SQL
SELECT
  h.hizea_id,
  p.objid,
  p.ra,
  p.dec,
  p.run,
  p.rerun,
  p.camcol,
  p.field,
  p.type,
  p.petroflux_u,
  p.petroflux_g,
  p.petroflux_r,
  p.petroflux_i,
  p.petroflux_z,
  p.petrofluxivar_u,
  p.petrofluxivar_g,
  p.petrofluxivar_r,
  p.petrofluxivar_i,
  p.petrofluxivar_z,
  p.modelflux_u,
  p.modelflux_g,
  p.modelflux_r,
  p.modelflux_i,
  p.modelflux_z,
  p.modelfluxivar_u,
  p.modelfluxivar_g,
  p.modelfluxivar_r,
  p.modelfluxivar_i,
  p.modelfluxivar_z,
  p.cmodelflux_u,
  p.cmodelflux_g,
  p.cmodelflux_r,
  p.cmodelflux_i,
  p.cmodelflux_z,
  p.cmodelfluxivar_u,
  p.cmodelfluxivar_g,
  p.cmodelfluxivar_r,
  p.cmodelfluxivar_i,
  p.cmodelfluxivar_z,
  p.extinction_u,
  p.extinction_g,
  p.extinction_r,
  p.extinction_i,
  p.extinction_z,
  p.petroR50_u,
  p.petroR50_g,
  p.petroR50_r,
  p.petroR50_i,
  p.petroR50_z,
  f.w1_nanomaggies,
  f.w1_nanomaggies_ivar,
  f.w2_nanomaggies,
  f.w2_nanomaggies_ivar
INTO mydb.hst_sample_sdssWISEphot from mydb.hizea_match as h
  JOIN dr14.PhotoObj AS p ON h.objid=p.objid
  JOIN dr14.wiseForcedTarget AS f ON h.objid=f.objid
ORDER BY h.hizea_id
```